<a href="https://colab.research.google.com/github/Amanj5486/Finance_ML/blob/main/Walmart_Tf_basedon_last6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import math
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import confusion_matrix, accuracy_score


In [23]:
df=pd.read_csv('WMT.csv',index_col='Date',parse_dates=True)

In [24]:
data=df.filter(['Adj Close'])
dataset=data.values
training_data_len=math.ceil(len(dataset)*0.8)
dataset

array([[ 41.53619 ],
       [ 41.765118],
       [ 41.2994  ],
       ...,
       [139.080002],
       [139.880005],
       [140.75    ]])

In [25]:
scaler=MinMaxScaler(feature_range=(0,1))
scaled_data=scaler.fit_transform(dataset)
scaled_data

array([[0.02707786],
       [0.02911245],
       [0.0249734 ],
       ...,
       [0.89399463],
       [0.90110462],
       [0.90883667]])

In [26]:
train_data=scaled_data[0:training_data_len,:]
x_train=[]
y_train=[]
for i in range(60,len(train_data)):
  x_train.append(train_data[i-60:i,0])
  y_train.append(train_data[i,0])

In [27]:
x_train,y_train=np.array(x_train),np.array(y_train)
x_train=np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))
x_train.shape

(1954, 60, 1)

In [28]:
model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(x_train.shape[1],1)))
model.add(LSTM(50,return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

In [29]:
model.compile(optimizer='adam',loss='mean_squared_error')

In [30]:
model.fit(x_train,y_train,batch_size=1,epochs=1)

1954/1954 [==============================] - 52s 25ms/step - loss: 0.0013


In [31]:
test_data=scaled_data[training_data_len-60:,:]
x_test=[]
y_test=dataset[training_data_len:,:]
for i in range(60,len(test_data)):
  x_test.append(test_data[i-60:i,0])

In [32]:
x_test=np.array(x_test)

In [33]:
x_test=np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))

In [34]:
predictions=model.predict(x_test)
predictions=scaler.inverse_transform(predictions)


In [35]:
rmse=np.sqrt(np.mean(predictions-y_test)**2)
rmse

3.904916533000241

In [36]:
valid=data[training_data_len:]
valid['Predictions']=predictions

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [37]:
valid

,Adj Close,Predictions
Date,,
2019-06-14,105.421082,101.258995
2019-06-17,105.508080,101.810394
2019-06-18,105.981689,102.224091
2019-06-19,105.952682,102.584023
2019-06-20,106.629257,102.849770
...,...,...
2021-06-07,140.899994,136.990555
2021-06-08,139.830002,136.834183
2021-06-09,139.080002,136.476425


In [38]:
r2=sklearn.metrics.r2_score(y_test, predictions)
r2

0.8779611078432885